# Shopify Data Science Internship Challenge

## Question 1

### a. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.

#### I decided to use the pandas library in Python for this question since it makes it simple to manipulate excel files:

In [1]:
#import pandas and numpy to visualize and analyze our dataset
import pandas as pd 
import numpy as np

#### My first thought is that it could be a calculation error, so I went back to the data to verify the value with my own calculation. The first step is to load and view the data

In [2]:
#Save our data into a dataframe and view
df = pd.read_excel('data.xlsx')
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1.0,53.0,746.0,224.0,2.0,cash,2017-03-13 12:36:56.190
1,2.0,92.0,925.0,90.0,1.0,cash,2017-03-03 17:38:51.999
2,3.0,44.0,861.0,144.0,1.0,cash,2017-03-14 04:23:55.595
3,4.0,18.0,935.0,156.0,1.0,credit_card,2017-03-26 12:43:36.649
4,5.0,18.0,883.0,156.0,1.0,credit_card,2017-03-01 04:35:10.773


#### The data file contains 7 unique features with 5000 data points. We are also provided with the date, so I'll quickly verify that the data indeed spans over a 30 day window.

In [3]:
#Sorts the values in the "created_at" column so we can see the full span of time data
df.sort_values(['created_at'])

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
1862,1863.0,39.0,738.0,536.0,4.0,cash,2017-03-01 00:08:09.179
1741,1742.0,39.0,910.0,268.0,2.0,cash,2017-03-01 00:10:19.043
3228,3229.0,97.0,912.0,324.0,2.0,cash,2017-03-01 00:14:12.250
1267,1268.0,80.0,798.0,290.0,2.0,credit_card,2017-03-01 00:19:31.258
2689,2690.0,49.0,799.0,258.0,2.0,credit_card,2017-03-01 00:22:24.790
...,...,...,...,...,...,...,...
2630,2631.0,53.0,940.0,112.0,1.0,credit_card,2017-03-30 23:12:13.085
1685,1686.0,34.0,818.0,244.0,2.0,cash,2017-03-30 23:16:09.573
1474,1475.0,21.0,815.0,142.0,1.0,cash,2017-03-30 23:26:54.436
317,318.0,52.0,848.0,292.0,2.0,cash,2017-03-30 23:41:34.347


#### So, from the above dataframe, we can see that the transactions are taken over 30 days in the month of March, which is good.

In [4]:
#Calculate our Average Order Value
AOV = df['order_amount'].mean()
AOV

3145.128

#### Looking at the Average Order Value (AOV), it does appear to be slightly higher than I would expect. After all, not a lot of people spend over $3,000 on shoes! This leads me to believe that there may be an outlier in the data causing a bias in the AOV. I'll use the pandas describe function to get a better understanding of the dataset

In [5]:
df['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

#### Wow, the median amount is \\$284, but the max is over \\$700k!! There's definitely a bias skewing the dataset, but now I need to find the source of the bias

#### Going back to the basics, the formula for Revenue is given by Revenue = Quantity x Price

#### Because the AOV is essentially a measure of the average revenue, we should be able to trace a bias in the revenue back to either the quantity, the price, or both.

#### Looking at the original dataset, we have data for both the revenue and the quantity, but not the price per item. However, we can reverse-engineer this feature using the Revenue formula

In [6]:
#First, lets drop the features which are no longer useful to us
df = df.drop(['order_id','created_at','payment_method'], axis=1)
df

,shop_id,user_id,order_amount,total_items
0,53.0,746.0,224.0,2.0
1,92.0,925.0,90.0,1.0
2,44.0,861.0,144.0,1.0
3,18.0,935.0,156.0,1.0
4,18.0,883.0,156.0,1.0
...,...,...,...,...
4995,73.0,993.0,330.0,2.0
4996,48.0,789.0,234.0,2.0
4997,56.0,867.0,351.0,3.0
4998,60.0,825.0,354.0,2.0


In [7]:
#Implementing the revenue formula to calculate the cost per item and saving the result into a new feature column
df['cost_per_item'] = df['order_amount']/df['total_items']
df

,shop_id,user_id,order_amount,total_items,cost_per_item
0,53.0,746.0,224.0,2.0,112.0
1,92.0,925.0,90.0,1.0,90.0
2,44.0,861.0,144.0,1.0,144.0
3,18.0,935.0,156.0,1.0,156.0
4,18.0,883.0,156.0,1.0,156.0
...,...,...,...,...,...
4995,73.0,993.0,330.0,2.0,165.0
4996,48.0,789.0,234.0,2.0,117.0
4997,56.0,867.0,351.0,3.0,117.0
4998,60.0,825.0,354.0,2.0,177.0


#### Now that we have all our features, we need to check if there are any outliers biasing the data. To do this, I can filter the data using its z-score. We'll start with the total items feature by filtering the transactions over 1 standard deviation above the mean.

In [8]:
#Calculating Z-score for the "total_items" feature and filtering the transactions over 1 standard deviation above the mean
df['zscore total_items'] = (df['total_items'] - df['total_items'].mean())/df['total_items'].std()
df[df['zscore total_items']>1]

,shop_id,user_id,order_amount,total_items,cost_per_item,zscore total_items
15,42.0,607.0,704000.0,2000.0,352.0,17.118357
60,42.0,607.0,704000.0,2000.0,352.0,17.118357
520,42.0,607.0,704000.0,2000.0,352.0,17.118357
1104,42.0,607.0,704000.0,2000.0,352.0,17.118357
1362,42.0,607.0,704000.0,2000.0,352.0,17.118357
1436,42.0,607.0,704000.0,2000.0,352.0,17.118357
1562,42.0,607.0,704000.0,2000.0,352.0,17.118357
1602,42.0,607.0,704000.0,2000.0,352.0,17.118357
2153,42.0,607.0,704000.0,2000.0,352.0,17.118357
2297,42.0,607.0,704000.0,2000.0,352.0,17.118357


#### Looks like shop 42 is the culprit! Over the course of 1 month, it sold 17 orders of 2000 shoes. This is over 17 standard deviations above the mean, which absolutely explains why we were getting such a high AOV! It looks like this shop sells in wholesale while the rest of the sneaker shops sell to individual consumers, judging by the total items sold. However, it could also be used for more nefarious purposes like money laundering, seeing as all the orders are made by the same user in the same quantities. We should flag the shop and user for further investigation.

#### We aren't done quite yet. We still need to do the same for the cost per item feature to ensure that it doesn't have any biases.

In [9]:
#Calculating Z-score for the "cost_per_item" feature and filtering the transactions over 1 standard deviation above the mean
df['zscore cost_per_item'] = (df['cost_per_item'] - df['cost_per_item'].mean())/df['cost_per_item'].std()
df[df['zscore cost_per_item']>1]

,shop_id,user_id,order_amount,total_items,cost_per_item,zscore total_items,zscore cost_per_item
160,78.0,990.0,25725.0,1.0,25725.0,-0.066946,10.375771
490,78.0,936.0,51450.0,2.0,25725.0,-0.058349,10.375771
493,78.0,983.0,51450.0,2.0,25725.0,-0.058349,10.375771
511,78.0,967.0,51450.0,2.0,25725.0,-0.058349,10.375771
617,78.0,760.0,51450.0,2.0,25725.0,-0.058349,10.375771
691,78.0,878.0,154350.0,6.0,25725.0,-0.023961,10.375771
1056,78.0,800.0,25725.0,1.0,25725.0,-0.066946,10.375771
1193,78.0,944.0,25725.0,1.0,25725.0,-0.066946,10.375771
1204,78.0,970.0,25725.0,1.0,25725.0,-0.066946,10.375771
1259,78.0,775.0,77175.0,3.0,25725.0,-0.049752,10.375771


#### Looks like we found another anomaly - shop 78 priced its sneakers at over 10 standard deviations above the mean! This time, it seems that this store sells a luxury sneaker, priced at $25,725. There's nothing too suspicious about this shop, as the users buying the shoes seem to appear normal. This store likely caters to luxury consumers.

#### Now that we identified the anomalies, it's time to think of a better way to represent this metric:

#### Option 1: Remove the outliers and re-calculate

In [10]:
#Calculate the AOV again, but this time removing all the outliers
AOV = df[(df['zscore cost_per_item']<1) & (df['zscore total_items']<1)]['order_amount'].mean()
AOV

302.58051448247926

#### Option 2: Calculate the median without removing outliers

In [11]:
#Calcualte the MOV without removing any outliers
MOV = df['order_amount'].median()
MOV

284.0

#### Option 3: Calculate the AOV using the median values of the quantity and price via the Revenue formula

In [12]:
AOV3 = df['cost_per_item'].median()*df['total_items'].median()
AOV3

306.0

### b. What metric would you report for this dataset?

#### We could still use Option 1 if we were willing to drop the outliers in the dataset. However, that may be unfavorable if we want to retain all the data, as dropping some data would not give us the entire picture.Option 3 is also possible, however it is the least accurate metric among the three methods, as it a sort round-a-bout way of achieving the answer. We could also calculate the Median Order Value (MOV) over the same 30 day window which would calculate the metric based on percentile instead. This would allow us to still include the outliers without introducing a bias in the final metric

### c. What is its value?

#### Median Order Value (MOV) = $284.0

#### This is far more reasonable than the initial average order value of over $3000. It should also be noted that an order can contain multiple shoes and therefore this metric should not be used to evaluate how affordable the the sneakers on Shopify are.